* This is a **WIP colab**.
* TODO:
  - Add a graph comparing sizes at different sparsity levels.

# Sparsity

This colab gives an intuition behind how sparsity improves model compression.

In [1]:
%%capture
import gzip
import operator, random

import numpy as np
import tensorflow as tf

from functools import reduce
from matplotlib import pyplot as plt

## Sparsity improves Model Compression
We use a randomly generated sample for this demonstration. We evaluate and compare the compression ratios achieved without and with sparsification. The expected outcome is higher compression ratio for the sparse model.

In [36]:
# Sparsify the weights by setting a fraction of the weights to zero.
def sparsify_smallest(w, sparsity_rate):
  w = w.copy()
  w_1d = np.reshape(w, (-1))
  
  # Create a list of indices that are sorted by the absolute magnitude of the
  # weights.
  w_1d_sorted_indices = np.argsort(np.abs(w_1d))

  # Compute the number of elements to zero.
  num_elements_to_zero = int(w_1d.shape[0] * sparsity_rate)

  # Set the respective indices to zero.
  w_1d[w_1d_sorted_indices[:num_elements_to_zero]] = 0.0

  w = np.reshape(w_1d, w.shape)
  return w

def compress_and_save(w):
  # Compress the weights matrix using gzip.
  compressed_w = gzip.compress(w.tobytes())
  return compressed_w

weights = np.random.normal(size=(100, 100)).astype(np.float32)
sparsity_rate = 0.4 # The percentage of weights that are zeroed out.

sparse_weights = sparsify_smallest(weights, sparsity_rate)

print('Original Size:', reduce(operator.mul, weights.shape)*weights.itemsize)

weights_compressed = compress_and_save(weights)
print('Original Compressed Size:', len(weights_compressed))

weights_sparsified_compressed = compress_and_save(sparse_weights)
print('Sparsified Compressed Size:', len(weights_sparsified_compressed))

Original Size: 40000
Original Compressed Size: 37103
Sparsified Compressed Size: 24844
